In [5]:
from joblib import Memory
from dask.distributed import Client, LocalCluster, wait, as_completed
import time
from tqdm import tqdm_notebook as tqdm

### Parallel Helper

In [1]:
# We wrap the clients in a class to prevent
# double init
class MyClient:
    mp = None
    mt = None
    modules = None 
    started = False

    # Stats
    p_workers = None 
    p_cores   = None 
    p_workers = None 
    p_cores   = None 
    
    @staticmethod
    def restart():
        MyClient.mp.restart()
        MyClient.mt.restart()
    
    @staticmethod
    def init():
        if MyClient.started is False:
            # Start process cluster
            print("Starting multiprocess cluster...")
            MyClient.mp = Client(scheduler_port=0, dashboard_address=None)
            MyClient.p_workers = len(MyClient.mp.cluster.workers)
            MyClient.p_cores = MyClient.mp.cluster.worker_kwargs['ncores']
            print("Multiprocess cluster started w/ {} workers ({} cores each)".format(MyClient.p_workers, MyClient.p_cores))
            
            # Start thread cluster
            print("Starting multithread cluster...")
            MyClient.mt = Client(scheduler_port=0, dashboard_address=None, processes=False)
            MyClient.t_workers = len(MyClient.mt.cluster.workers)
            MyClient.t_cores = MyClient.mt.cluster.worker_kwargs['ncores']
            print("Multithread cluster started w/ {} workers ({} cores each)".format(MyClient.t_workers, MyClient.t_cores))
            
            # Toggle             
            MyClient.started = True
        else:
            print("Multithread cluster already started")

def init_parallel(module_names=None):
    MyClient.init()    
    if module_names is not None:
        for name in module_names:
            MyClient.mp.upload_file(name)
            MyClient.mt.upload_file(name)
    print(
        """
        Parallel Plugin Loaded. You can now decorate functions with @profile(profile_array) 
        and @parallel(map=True, threads=True, background=False). MyClient and get_results(futures)
        have also been loaded into your namespace.
        """)
    
def parallel(map=False, threads=False, background=True):
    def concurrent_decorator(func):
        def wrapper_concurrent(*args, **kwargs):
            client = MyClient.mt if threads else MyClient.mp
            if map:
                res = client.map(func, *args, **kwargs)
            else:
                res = client.submit(func, *args, **kwargs)
            if background:
                return res
            else:
                return [r for f, r in tqdm(as_completed(res, with_results=True), total=len(res))]
        return wrapper_concurrent
    return concurrent_decorator

def get_results(futures):
    return [f.result() for f in as_completed(futures)]